In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

In [2]:
energy = pd.read_csv("datasets/energy_dataset.csv")

In [3]:
energy.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [4]:
energy["time"] = pd.to_datetime(energy["time"], utc=True)

In [5]:
energy["day"] = energy["time"].dt.day
energy["month"] = energy["time"].dt.month
energy["hour"] = energy["time"].dt.hour
energy["day_of_week"] = energy["time"].dt.day_of_week

In [6]:
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3,4,5]:
        return "Spring"
    elif month in [6,7,8]:
        return "Fall"
    else:
        return "Winter"
    
energy["season"] = energy["month"].apply(get_season)   


In [7]:
energy.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,day,month,hour,day_of_week,season
0,2014-12-31 23:00:00+00:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,6436.0,26118.0,25385.0,50.10,65.41,31,12,23,2,Winter
1,2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,5856.0,24934.0,24382.0,48.10,64.92,1,1,0,3,Winter
2,2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,5454.0,23515.0,22734.0,47.33,64.48,1,1,1,3,Winter
3,2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,5151.0,22642.0,21286.0,42.27,59.32,1,1,2,3,Winter
4,2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,4861.0,21785.0,20264.0,38.41,56.04,1,1,3,3,Winter


In [8]:
energy.isna().sum()

time                                               0
generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                                17
generation other                                  18
generation other renewable                        18
generation solar                              

In [9]:
for column in energy.columns:
    if energy[column].isnull().sum() / len(energy) < 0.05 and energy[column].isnull().sum() / len(energy) > 0:
        energy[column].fillna(energy[column].mean(), inplace=True)

C:\Users\aakas\AppData\Local\Temp\ipykernel_18936\3254980725.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  energy[column].fillna(energy[column].mean(), inplace=True)


In [10]:
energy.dropna(axis=1, how='all', inplace=True)

In [11]:
energy.isna().sum()

time                                           0
generation biomass                             0
generation fossil brown coal/lignite           0
generation fossil coal-derived gas             0
generation fossil gas                          0
generation fossil hard coal                    0
generation fossil oil                          0
generation fossil oil shale                    0
generation fossil peat                         0
generation geothermal                          0
generation hydro pumped storage consumption    0
generation hydro run-of-river and poundage     0
generation hydro water reservoir               0
generation marine                              0
generation nuclear                             0
generation other                               0
generation other renewable                     0
generation solar                               0
generation waste                               0
generation wind offshore                       0
generation wind onsh

In [12]:
energy.dropna(inplace=True)

In [13]:
energy.set_index("time", inplace=True)

for lag in range(1,8):
    energy[f"total_load_actual_lag{lag}"] = energy["total load actual"].shift(lag)

energy_daily = energy.resample('D').sum()

In [14]:
for lag in range(1, 8):
    energy_daily[f"total_load_actual_lag{lag}"] = energy_daily["total load actual"].shift(lag)
 
energy_weekly = energy.resample("W").sum()

In [15]:
for lag in range(1, 8):
    energy_weekly[f'total_load_actual_lag{lag}'] = energy_weekly['total load actual'].shift(lag)

In [16]:
energy["hour"] = energy.index.hour
energy["day"] = energy.index.day
energy["month"] = energy.index.month

In [23]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [17]:
features = [
    'generation solar', 'generation wind onshore', 'forecast solar day ahead', 
    'price day ahead', 'hour', 'day', 'month'
] + [f'total_load_actual_lag{lag}' for lag in range(1, 8)]
target = 'total load actual'

In [24]:
X = energy[features]
y = energy[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

In [34]:
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror')
xgb_model.fit(X_train, y_train)

# Predict
y_pred_xgb = xgb_model.predict(X_test)

In [35]:
arima_model = ARIMA(y_train, order=(5, 1, 0))
arima_model_fit = arima_model.fit()

# Predict
y_pred_arima = arima_model_fit.forecast(steps=len(y_test))

c:\Users\aakas\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\aakas\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\aakas\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\aakas\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not mo

In [28]:
import numpy as np

def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    return rmse, mae, mape

# Calculate metrics for Random Forest
rmse_rf, mae_rf, mape_rf = calculate_metrics(y_test, y_pred_rf)
print(f'Random Forest - RMSE: {rmse_rf}, MAE: {mae_rf}, MAPE: {mape_rf}')

# Calculate metrics for XGBoost
rmse_xgb, mae_xgb, mape_xgb = calculate_metrics(y_test, y_pred_xgb)
print(f'XGBoost - RMSE: {rmse_xgb}, MAE: {mae_xgb}, MAPE: {mape_xgb}')

# Calculate metrics for ARIMA
rmse_arima, mae_arima, mape_arima = calculate_metrics(y_test, y_pred_arima)
print(f'ARIMA - RMSE: {rmse_arima}, MAE: {mae_arima}, MAPE: {mape_arima}')

Random Forest - RMSE: 503.6003494037412, MAE: 276.76395934320965, MAPE: 0.009821863160540758
XGBoost - RMSE: 6384.558630134659, MAE: 5202.624018349703, MAPE: 0.18796437483167705
ARIMA - RMSE: 7045.745043734059, MAE: 5756.379069359932, MAPE: 0.1841358208263243


In [32]:
# Define cross-validation method
cv = 5

# Calculate cross-validated RMSE
cv_rmse_rf = np.sqrt(-cross_val_score(rf_model, X, y, scoring='neg_mean_squared_error', cv=cv))
cv_mae_rf = -cross_val_score(rf_model, X, y, scoring='neg_mean_absolute_error', cv=cv)
cv_mape_rf = -cross_val_score(rf_model, X, y, scoring='neg_mean_absolute_percentage_error', cv=cv)

print(f'Random Forest - CV RMSE: {cv_rmse_rf.mean()}, CV MAE: {cv_mae_rf.mean()}, CV MAPE: {cv_mape_rf.mean()}')


KeyboardInterrupt: 

In [36]:
# Calculate cross-validated RMSE
cv_rmse_xgb = np.sqrt(-cross_val_score(xgb_model, X, y, scoring='neg_mean_squared_error', cv=cv))
cv_mae_xgb = -cross_val_score(xgb_model, X, y, scoring='neg_mean_absolute_error', cv=cv)
cv_mape_xgb = -cross_val_score(xgb_model, X, y, scoring='neg_mean_absolute_percentage_error', cv=cv)

print(f'XGBoost - CV RMSE: {cv_rmse_xgb.mean()}, CV MAE: {cv_mae_xgb.mean()}, CV MAPE: {cv_mape_xgb.mean()}')


XGBoost - CV RMSE: 539.6198213643095, CV MAE: 331.66229267758644, CV MAPE: 0.011813726011902633
